In [ ]:
import folium
import hvplot.pandas
import panel as pn
from bokeh.plotting import figure

import garmin_viz

pn.extension("tabulator")

In [ ]:
# load gpx data into a DataFrame
activity_name, df_raw = garmin_viz.load_activity("data/activity_9966616577.gpx")
# df_raw.head()

In [ ]:
# data engineering: 
df = garmin_viz.extend_dataframe(df_raw)
# df.head()

In [ ]:
# preprocessing
df_clean = garmin_viz.remove_stop_segments(df)
# df_clean.describe()

In [ ]:
# Compute some statistics

# TODO: Date and time
date = df_clean["time"].iloc[0]

# Total distance
d_tot = df_clean["distance_2d"].iloc[-1] # in meters
d_tot /= 1000.0  # in km

# Elapsed time
t = df_clean["time"].iloc[-1] - df_clean["time"].iloc[0]
s = t.seconds
hours, remainder = divmod(s, 3600)
minutes, seconds = divmod(remainder, 60)
elapsed_time = f"{int(hours)}:{int(minutes)}:{int(seconds)}"

# Pace
# convert from second per meter to minutes per km
avg_pace = df_clean["pace"].mean() * (1000.0 / 60.0)  # min/km
avg_pace_str = garmin_viz.convert_pace_to_str(avg_pace)  # strava: 5:24 /km

# Heart rate (beats per minute)
avg_hr = round(df_clean["hr"].mean())  # strava: 168 bpm

# Cadence (steps per minute)
avg_cadence = round(df_clean["cadence"].mean())  # strava: 172 spm


# Messages for the dashboard
date_msg = date.strftime("%b %d, %Y at %H:%M")
total_distance_msg = f"Total distance: {d_tot:.1f} km"
elapsed_time_msg = f"Elapsed time: {elapsed_time}"
avg_pace_msg = f"Pace: {avg_pace_str} / km"
avg_hr_msg = f"Heart rate: {avg_hr} bpm"
avg_cadence_msg = f"Cadence: {avg_cadence} spm"

# Test
# print(total_distance_msg)
# print(elapsed_time_msg)
# print(avg_pace_msg)
# print(avg_hr_msg)
# print(avg_cadence_msg)


Dashboard

In [ ]:
stats_msg = f"""
# {date_msg} 

# Statistics


### Total distance: *{d_tot:.1f} km*

### Elapsed time: *{elapsed_time}*

### Pace: *{avg_pace_str} / km*

### Heart rate: *{avg_hr} bpm*


### Cadence: *{avg_cadence} spm*
"""
# stats_msg

In [ ]:
# display course on map
centroid = (df_clean["latitude"].mean(), df_clean["longitude"].mean())
course_map = folium.Map(location=centroid, zoom_start=12)
folium.PolyLine(
    df_clean[["latitude", "longitude"]],
    color="#fc4c02").add_to(course_map);
# pn.panel(course_map, height=400)

In [ ]:
# bin data into 1km laps
split_1k = garmin_viz.bin_data(df_clean, 1.0)
# split_1k.head()

In [ ]:
split_pipeline = split_1k.set_index("lap").interactive()

In [ ]:
# split plot
# yaxis_split_plot = pn.widgets.MultiChoice(
#     name='MultiSelect', value=["pace"], options=["pace", "cadence", "hr"]
# )
# split_plot = split_pipeline.hvplot(x = 'lap', y=yaxis_split_plot,line_width=2, title="Lap")

In [ ]:
# pretty_pace_plot = garmin_viz.pretty_pace_plot(split_1k["lap"], split_1k["pace_float"])

In [ ]:
p1 = figure(height=300, width=1300, sizing_mode='stretch_width')
p2 = figure(height=300, width=1300, sizing_mode='stretch_width')
p3 = figure(height=300, width=1300, sizing_mode='stretch_width')


p1.line(split_1k["lap"], split_1k["pace_float"])
p1.dash(split_1k["lap"], split_1k.shape[0] * [avg_pace])
p2.line(split_1k["lap"], split_1k["hr"])
p2.dash(split_1k["lap"], split_1k.shape[0] * [avg_hr])
p3.line(split_1k["lap"], split_1k["cadence"])
p3.dash(split_1k["lap"], split_1k.shape[0] * [avg_cadence])

p1.yaxis.axis_label = "pace [min/km]"
p2.yaxis.axis_label = "hear rate [bpm]"
p3.yaxis.axis_label = "cadence [spm]"

p3.xaxis.axis_label = "lap (km)"

In [ ]:
# split table
split_table = split_pipeline[["pace", "hr", "cadence"]].pipe(
    pn.widgets.Tabulator, pagination="remote", page_size=20, # sizing_mode="stretch_width"
)

In [ ]:
# dashboard
template = pn.template.FastListTemplate(
    title=activity_name,
    sidebar=[pn.pane.Markdown(stats_msg)],
    main=[
        pn.Row(
            split_table.panel(width=300),
            pn.Column(pn.panel(course_map, height=600, width=1000))),
        # pn.Row(yaxis_split_plot, split_plot.panel(width=1200)),
        pn.Row(pn.Column(p1, p2, p3)),
    ],
    accent_base_color="#fc4c02",
    header_background="#fc4c02",
)

# template.show()
template.servable()
